## Group Level Analysis

This notebook is designed to perform a voxelwise analysis of the infant CBF data

In [6]:
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.fsl.utils import Merge, Split
from nipype.interfaces.fsl.model import Randomise, Cluster, GLM
from nipype.interfaces.fsl.maths import ApplyMask

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

#other study-specific variables
#project_home = '/Volumes/iang/active/BABIES/BABIES_perfusion'
project_home = '/Users/catcamacho/Box/SNAP/BABIES/BABIES_asl'
template_dir = '/Users/catcamacho/Box/SNAP/BABIES/templates'
output_dir = project_home + '/proc/asl_group_4factor'
preproc_dir = project_home + '/proc/asl_preproc'
wkflow_dir = project_home + '/workflows'
template = template_dir + '/6mo_T2w_template_2mm.nii.gz'
mask = template_dir + '/6mo_T2w_template_2mm_gm.nii.gz'

# Files for group level analysis
group_mat = project_home + '/misc/design_4factor.mat'
t_contrasts = project_home + '/misc/tcon_4factor.con'

subjects_list = open(project_home + '/misc/subjects_asl_temp.txt').read().splitlines()

In [7]:
# Data Handling Nodes

datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir

grabcbfdata = Node(DataGrabber(template=preproc_dir + '/cbf_volume/*/salsop_cbf_masked.nii.gz', 
                               sort_filelist=True, 
                               outfields=['cbf_list']), 
                   name='grabcbf')


In [8]:
def get_cluster_peaks(clusters_file, stat_file):
    from nibabel import load, save, Nifti1Image
    from pandas import DataFrame, Series
    from numpy import unique, unravel_index, max
    from os.path import abspath
    
    # load up clusters
    clusters_nii = load(clusters_file)
    clusters_data = clusters_nii.get_data()
    cluster_labels, cluster_sizes = unique(clusters_data, return_counts=True)
    cluster_sizes = cluster_sizes[cluster_labels>0]
    cluster_labels = cluster_labels[cluster_labels>0]
    
    # set up dataframe
    cluster_info = DataFrame(columns=['clust_num','peak','num_voxels','X','Y','Z'])
    cluster_info['clust_num'] = Series(cluster_labels,index=None)
    
    for i in range(0,len(cluster_labels)):
        # load up stat image
        stat_nii = load(stat_file)
        stat_data = stat_nii.get_data()
        stat_data[clusters_data!=cluster_labels[i]]=0
        location=unravel_index(stat_data.argmax(), stat_data.shape)
        cluster_info.iloc[i,0]=cluster_labels[i]
        cluster_info.iloc[i,1]=max(stat_data)
        cluster_info.iloc[i,2]=cluster_sizes[i]
        cluster_info.iloc[i,3]=location[0]
        cluster_info.iloc[i,4]=location[1]
        cluster_info.iloc[i,5]=location[2]
    
    out_prefix = clusters_file[:-7]
    cluster_info.to_csv(out_prefix + '_peaks.csv')
    cluster_info_file = abspath(out_prefix + '_peaks.csv')
    return(cluster_info_file)

def extract_cluster_betas(cluster_index_file, sample_betas, min_clust_size, subject_ids):
    from nibabel import load, save, Nifti1Image
    from pandas import DataFrame, Series
    from numpy import unique, zeros_like, invert
    from nipype.interfaces.fsl.utils import ImageMeants
    from os.path import abspath, basename
    
    sample_data = DataFrame(subject_ids, index=None, columns=['Subject'])
    
    cluster_nifti = load(cluster_index_file)
    cluster_data = cluster_nifti.get_data()
    clusters, cluster_sizes = unique(cluster_data, return_counts=True)
    cluster_sizes = cluster_sizes[clusters>0]
    clusters = clusters[clusters>0]
    clusters = clusters[cluster_sizes>min_clust_size]
    cluster_sizes = cluster_sizes[cluster_sizes>min_clust_size]
    ind_filename = basename(cluster_index_file) 
    out_prefix = ind_filename[:-7]
    
    for clust_idx in clusters:
        temp = zeros_like(cluster_data)
        temp[cluster_data==clust_idx] = 1
        temp_nii = Nifti1Image(temp,cluster_nifti.affine)
        temp_file = 'temp_clust_mask.nii.gz'
        save(temp_nii, temp_file)

        eb = ImageMeants()
        eb.inputs.in_file = sample_betas
        eb.inputs.mask = temp_file
        eb.inputs.out_file = 'betas.txt'
        eb.run()
        betas = open('betas.txt').read().splitlines()
        sample_data['clust' + str(clust_idx)] = Series(betas, index=sample_data.index)
    
    sample_data.to_csv(out_prefix+'_extracted_betas.csv')
    extracted_betas_csv = abspath(out_prefix+'_extracted_betas.csv')

    return(extracted_betas_csv)

In [9]:
# Analysis Nodes

merge = Node(Merge(dimension = 't'), name = 'merge')

apply_mask = Node(ApplyMask(mask_file=mask, nan2zeros=True), name='apply_mask')

model = Node(GLM(design=group_mat, mask=mask, 
                 out_p_name='pvals.nii.gz',
                 out_t_name='tstat.nii.gz', 
                 contrasts=t_contrasts), 
             name='model')

split = Node(Split(dimension='t'), name='split')

cluster = MapNode(Cluster(out_localmax_txt_file = 'cluster_stats.txt',
                          threshold=3.3256, 
                          dlh=4, 
                          pthreshold=0.01, 
                          volume=113120,
                          out_index_file='clusters.nii.gz'), 
                  name='cluster', iterfield=['in_file'])

get_peaks = MapNode(Function(input_names=['clusters_file', 'stat_file'], 
                             output_names=['cluster_info_file'], 
                             function=get_cluster_peaks), 
                    name='get_peaks', iterfield=['clusters_file', 'stat_file'])

get_betas = MapNode(Function(input_names=['cluster_index_file', 'sample_betas', 
                                          'min_clust_size', 'subject_ids'], 
                             output_names=['extracted_betas_csv'], 
                             function=extract_cluster_betas), 
                    name='get_betas', iterfield=['cluster_index_file'])
get_betas.inputs.subject_ids = subjects_list
get_betas.inputs.min_clust_size=10

In [10]:
# Analysis workflow

grouplevel = Workflow(name='grouplevel_4factor')

grouplevel.connect([(grabcbfdata, merge,[('cbf_list', 'in_files')]),
                    (merge, model, [('merged_file','in_file')]),
                    
                    (model, split, [('out_t','in_file')]),
                    (split, cluster, [('out_files','in_file')]),
                    (cluster, get_peaks, [('index_file','clusters_file')]),
                    (split, get_peaks, [('out_files','stat_file')]),
                    (cluster, get_betas ,[('index_file','cluster_index_file')]),
                    (merge, get_betas, [('merged_file','sample_betas')]),
                    
                    (get_peaks, datasink, [('cluster_info_file','affect_cluster_stats')]),
                    (get_betas, datasink, [('extracted_betas_csv','affect_cluster_betas')]),
                    (model, datasink, [('out_p','pval_files'),
                                       ('out_t','tstat_files')]),
                    (cluster, datasink, [('index_file','clusters_file')])
                   ])

grouplevel.base_dir = wkflow_dir
#grouplevel.write_graph(graph2use='flat')
grouplevel.run('MultiProc', plugin_args={'n_procs': 2})


190414-15:41:41,356 nipype.workflow INFO:
	 Workflow grouplevel_4factor settings: ['check', 'execution', 'logging', 'monitoring']
190414-15:41:41,388 nipype.workflow INFO:
	 Running in parallel.
190414-15:41:41,392 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 2/2.
190414-15:41:41,440 nipype.workflow INFO:
	 [Node] Setting-up "grouplevel_4factor.grabcbf" in "/Users/catcamacho/Box/SNAP/BABIES/BABIES_asl/workflows/grouplevel_4factor/grabcbf".
190414-15:41:41,479 nipype.workflow INFO:
	 [Node] Running "grabcbf" ("nipype.interfaces.io.DataGrabber")
190414-15:41:41,547 nipype.workflow INFO:
	 [Node] Finished "grouplevel_4factor.grabcbf".
190414-15:41:43,396 nipype.workflow INFO:
	 [Job 0] Completed (grouplevel_4factor.grabcbf).
190414-15:41:43,402 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 2/2.
190414-15:41:43,483 nipype.workflow INFO:
	 [Job 1

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import read_csv, DataFrame
from scipy.stats import pearsonr

sns.set(context='talk', style='white')

data = read_csv('/Users/catcamacho/Box/SNAP/BABIES/proc/asl_group_affect/affect_cluster_betas/_get_betas8/clusters_extracted_betas.csv', index_col=0)
clusts = data.columns
clusts = clusts[3:]
print(clusts)
for cluster in clusts:
    sns.jointplot(x='age',y=cluster,data=data,kind='reg')
    plt.xlabel('Infant Age (months)')
    plt.ylabel('Gray Matter CBF (mL/100g/min)')
    plt.savefig('/Users/catcamacho/Dropbox/Projects/infant_ASL/paper/strictagePlot_'+ cluster +'.svg')
    plt.show()
    print(pearsonr(data['age'],data[cluster]))



In [15]:
from pandas import read_csv
from scipy.stats import spearmanr
from numpy import round
file = '/Users/catcamacho/Box/SNAP/BABIES/BABIES_asl/results/5predictors.csv'
data=read_csv(file,index_col=None)
data.head()

rho, pval = spearmanr(data.values,nan_policy='omit')
print(data.columns)
print(round(rho,decimals=3))
print(round(pval,decimals=3))

Index(['mean_react_dis', 'mean_reg_pos', '3f_surgency', '3f_negaff',
       '3f_sensitivity', 'age'],
      dtype='object')
[[ 1.    -0.069 -0.009  0.    -0.245  0.001]
 [-0.069  1.     0.109 -0.136  0.012  0.11 ]
 [-0.009  0.109  1.    -0.06   0.393  0.19 ]
 [ 0.    -0.136 -0.06   1.    -0.275 -0.091]
 [-0.245  0.012  0.393 -0.275  1.     0.078]
 [ 0.001  0.11   0.19  -0.091  0.078  1.   ]]
[[0.    0.512 0.93  0.998 0.019 0.995]
 [0.512 0.    0.301 0.196 0.912 0.296]
 [0.93  0.301 0.    0.54  0.    0.05 ]
 [0.998 0.196 0.54  0.    0.004 0.353]
 [0.019 0.912 0.    0.004 0.    0.424]
 [0.995 0.296 0.05  0.353 0.424 0.   ]]


In [17]:
from pandas import read_csv
from scipy.stats import spearmanr
from numpy import round
file = '/Users/catcamacho/Box/SNAP/BABIES/BABIES_asl/results/5predictors.csv'
data=read_csv(file,index_col=None)
data.head()

rho, pval = spearmanr(data.values,nan_policy='omit')
print(data.columns)
print(round(rho,decimals=3))
print(round(pval,decimals=3))

Index(['mean_react_dis', 'mean_reg_pos', '4f_negaff', '4f_surgency',
       '4f_sensitivity', '4f_regulation', 'age'],
      dtype='object')
[[ 1.    -0.069  0.011 -0.035 -0.234 -0.02   0.001]
 [-0.069  1.    -0.185  0.12   0.025  0.067  0.11 ]
 [ 0.011 -0.185  1.     0.059 -0.21  -0.368 -0.033]
 [-0.035  0.12   0.059  1.     0.316  0.303  0.179]
 [-0.234  0.025 -0.21   0.316  1.     0.359  0.092]
 [-0.02   0.067 -0.368  0.303  0.359  1.     0.15 ]
 [ 0.001  0.11  -0.033  0.179  0.092  0.15   1.   ]]
[[0.    0.512 0.914 0.743 0.025 0.852 0.995]
 [0.512 0.    0.077 0.255 0.811 0.528 0.296]
 [0.914 0.077 0.    0.549 0.03  0.    0.736]
 [0.743 0.255 0.549 0.    0.001 0.002 0.065]
 [0.025 0.811 0.03  0.001 0.    0.    0.347]
 [0.852 0.528 0.    0.002 0.    0.    0.122]
 [0.995 0.296 0.736 0.065 0.347 0.122 0.   ]]


In [20]:
# Find the upper percentage of t-stats with respect to age
from nibabel import load
from numpy import nanpercentile

file = '/Users/catcamacho/Box/SNAP/BABIES/BABIES_asl/proc/asl_group_4factor/tstat_files/vol0012.nii.gz'

img =load(file)
data = img.get_data()
data = data[data>0]

print(nanpercentile(data, 95,interpolation='linear'))

32.18253135681152


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':
